In [ ]:
#importando bibliotecas
import os
import base64
import requests
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import numpy as np

#grafico
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Lendo os arquivos
#df_docente = pd.read_csv("DM_DOCENTE.CSV", sep="|")
df_aluno = pd.read_csv("DM_ALUNO.CSV", sep="|")
#df_variaveis = pd.read_excel("Dicionário de Variáveis.xls", "DM_ALUNO")
#df_codigo = pd.read_excel("Código_do_País_de_Nascimento_ou_Naturalização.xlsx")
#df_disciplinas = pd.read_csv("dados_disciplinas.txt", sep=",")

In [ ]:

chunksize = 1000

for chunk in pd.read_csv("DM_ALUNO.csv", , sep="|", chunksize=chunksize):
    # exemplo: contar NAN por coluna
    
    print(chunk)


In [ ]:
df_disciplinas[df_disciplinas['MEDIA_FINAL']==-1] 

In [ ]:
df_disciplinas['SITUACAO_NUM'] = (df_disciplinas['SITUACAO'] == 'Aprovado').astype(int)

In [ ]:
df_disciplinas_sem_equivalencia = df_disciplinas[df_disciplinas['SITUACAO'] != 'Equivalencia curricular']

In [ ]:
df_disciplinas_aprovado_ou_reprovado = df_disciplinas[(df_disciplinas['SITUACAO'] == 'Aprovado') | (df_disciplinas['SITUACAO'] == 'Reprovado')] 

In [ ]:
df_disciplinas_media_maior_zero = df_disciplinas[df_disciplinas['MEDIA_FINAL'] > 0]

In [ ]:
# plotar correlação alunos (Não funcionou, queria ver quais são as variáveis que podemos eliminar pela correlação)

corr = df_disciplinas_media_maior_zero.corr(numeric_only=True)

plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Matriz de Correlação")
plt.show()

In [ ]:
df_disciplinas_remove_media_negativa = df_disciplinas[ (df_disciplinas['MEDIA_FINAL'] != -1)] 

In [ ]:
# plotar correlação alunos (Não funcionou, queria ver quais são as variáveis que podemos eliminar pela correlação)

corr = df_disciplinas_remove_media_negativa.corr(numeric_only=True)

plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Matriz de Correlação")
plt.show()

In [ ]:
X = df_disciplinas_media_maior_zero[['MATR_ALUNO', 'ANO', 'PERIODO', 'MEDIA_FINAL', 'NUM_FALTAS', 'DISCIPLINA']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

dbscan = DBSCAN(eps=2, min_samples=50)
clusters = dbscan.fit_predict(X_scaled)

df_disciplinas_media_maior_zero['cluster_dbscan'] = clusters

pd.crosstab(df_disciplinas_media_maior_zero['cluster_dbscan'], df_disciplinas_media_maior_zero['SITUACAO_NUM'])

In [ ]:
X = df_disciplinas_remove_media_negativa[['MATR_ALUNO', 'ANO', 'PERIODO', 'MEDIA_FINAL', 'NUM_FALTAS', 'DISCIPLINA', 'SITUACAO_NUM']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

dbscan = DBSCAN(eps=1.5, min_samples=10)
clusters = dbscan.fit_predict(X_scaled)

df_disciplinas_remove_media_negativa['cluster_dbscan'] = clusters

pd.crosstab(df_disciplinas_remove_media_negativa['SITUACAO'], df_disciplinas_remove_media_negativa['cluster_dbscan'])

In [ ]:
plt.figure(figsize=(10, 6))

clusters = df_disciplinas_remove_media_negativa['cluster_dbscan'].unique()

for cluster in clusters:
    # Mask do cluster
    mask_cluster = df_disciplinas_remove_media_negativa['cluster_dbscan'] == cluster

    # Separar aprovados e reprovados dentro do cluster
    mask_ap = mask_cluster & (df_disciplinas_remove_media_negativa['SITUACAO'] == 'Aprovado')
    mask_rep = mask_cluster & (df_disciplinas_remove_media_negativa['SITUACAO'] == 'Reprovado')

    # Nome do cluster para legenda
    if cluster == -1:
        cluster_label = "Ruído (outliers)"
    else:
        cluster_label = f"Cluster {cluster}"

    # Plotar Aprovados (X)
    plt.scatter(
        df_disciplinas_remove_media_negativa.loc[mask_ap, 'MEDIA_FINAL'],
        df_disciplinas_remove_media_negativa.loc[mask_ap, 'NUM_FALTAS'],
        label=f"{cluster_label} - Aprovados",
        marker='x'
    )

    # Plotar Reprovados (O)
    plt.scatter(
        df_disciplinas_remove_media_negativa.loc[mask_rep, 'MEDIA_FINAL'],
        df_disciplinas_remove_media_negativa.loc[mask_rep, 'NUM_FALTAS'],
        label=f"{cluster_label} - Reprovados",
        marker='o'
    )

plt.xlabel("Nota Final")
plt.ylabel("Número de Faltas")
plt.title("Clusters DBSCAN com Aprovados (X) e Reprovados (O)")
plt.legend()
plt.grid(True)
plt.show()